In [ ]:
import pandas as pd
from tkinter import Tk
from tkinter.filedialog import askopenfilename

# Use tkinter to select the Excel file
def select_file():
    Tk().withdraw()
    file_path = askopenfilename(
        title="Select Training Excel File",
        filetypes=[("Excel files", "*.xlsx *.xls"), ("All files", "*.*")]
    )
    return file_path

# Prompt the user
print("Please select the training data file:")
training_excel_file = select_file()

if training_excel_file:
    # Read Excel file
    df_training = pd.read_excel(training_excel_file, sheet_name='Sheet1')
    
    print("Top 5 rows in the training DataFrame:")
    print(df_training.head())
else:
    print("No file selected. Please try again.")

**Data cleansing**

In [ ]:
import json

# Load materials.json
with open('../data/processed/materials.json', 'r') as file:
    materials_data = json.load(file)

# Convert materials.json into a lookup dictionary for efficient matching
materials_lookup = {item['material_number']: item['description'] for item in materials_data}

# Ensure the necessary columns exist
if 'Combined Description' not in df_training.columns or 'Commitment item' not in df_training.columns:
    raise ValueError("The training Excel file must contain 'Combined Description' and 'Commitment item' columns.")

# Data cleansing: Remove null values and strip whitespace
df_training = df_training.dropna(subset=['Combined Description', 'Commitment item'])
df_training['Combined Description'] = df_training['Combined Description'].str.strip()

# Extract material_number from 'Commitment item' and map to descriptions
df_training['material_number'] = df_training['Commitment item'].str.split(' - ', expand=True)[0]
df_training['description'] = df_training['material_number'].map(materials_lookup)

# Drop the original 'Commitment item' column
df_training = df_training.drop(columns=['Commitment item'])

# Display the cleaned DataFrame
print("\nCleaned training DataFrame:")
print(df_training[['material_number', 'description', 'Combined Description']].head())


**Vector embedding of materials.json**

In [ ]:
import sys
import os

sys.path.append(os.path.abspath(".."))
from src import search
json_file = '../data/processed/services.json'

# Initialize the search engine with reference data
search_engine = search.SentenceTransformerSearch(data_file=json_file)

print("\nVector embedding completed for reference data")

**Evaluate model, save to dataframe**

In [ ]:
from sklearn.model_selection import train_test_split
from src.search import evaluate_model

# Split the data 
train_df, test_df = train_test_split(df_training, test_size=0.5, random_state=42)

# Evaluate the model    
results = evaluate_model(search_engine, train_df, top_k=5)

# Convert the results to a DataFrame 
df_default = pd.DataFrame(results)
df = df_default
print("\nEvaluation completed. Results DataFrame:")
print(df.head(1))

## Analysis 2: which service numbers are performing well and why?

**for the top performers compute precision as well as the correct/incorrect predictions**<br>
How metrics are previously calculated:<br>
In the previous calculations, even if a record is classified as "correct" for one of the top 5 values matching the expected service number, any other service numbers present in the top 5 that are not the expected service number will contribute to their own incorrect count.<br>

This may not make much sense as it might inflate the model's incorrectness <br>

How they are calculated now:<br>
If in one record, any of the top 5 matches the expected, then it would not add to the incorrect count for the other 4 service numbers

In [ ]:
import pandas as pd
from IPython.display import display, HTML  

# Flatten the data into a DataFrame
df_copy = pd.DataFrame([
    {
        "query": entry["query"],
        "expected": entry["expected"],
        "expected_description": entry["expected_description"],  
        "retrieved_material_number": retrieved["material_number"],
        "retrieved_description": retrieved["description"],
        "score": retrieved["score"],
        "is_correct": retrieved["material_number"] == entry["expected"]
    }
    for entry in df.to_dict("records") 
    for retrieved in entry["retrieved_top_5"]
])

'''
Why True Negatives Don't Apply
In this context:

- The model is not tasked with determining what doesn’t belong.
- There is no explicit prediction or evaluation for items outside the top 5.
- Therefore, true negatives are not meaningful.
'''

# Exclude matches if any in the top 5 is correct
df_copy['query_correct'] = df_copy.groupby('query')['is_correct'].transform('max')

# Calculate metrics for each service number, including hit rate
service_metrics = []
for material_number, group in df_copy.groupby("retrieved_material_number"):
    total_correct = group["is_correct"].sum()
    total_incorrect = group[(group['query_correct'] == 0)].shape[0]
    false_negatives = len(df_copy[(df_copy["expected"] == material_number) &
                                  (~df_copy["retrieved_material_number"].eq(material_number))])

    precision = total_correct / (total_correct + total_incorrect) if (total_correct + total_incorrect) > 0 else 0
    accuracy = total_correct / len(group)
    #accuracy = total_correct / (total_correct + total_incorrect + false_negatives) if (total_correct + total_incorrect + false_negatives) > 0 else 0
    hit_rate = len(group)

    description = group["retrieved_description"].iloc[0]

    # Append metrics for this service number
    service_metrics.append({
        "service_number": material_number, 
        "hit_rate": hit_rate,
        "total_correct": total_correct,
        "total_incorrect": total_incorrect,
        "precision": round(precision, 3),  
        "accuracy": round(accuracy, 3),   
        "description": description
    })

metrics_df = pd.DataFrame(service_metrics)
metrics_df = metrics_df.sort_values(by="precision", ascending=False)

custom_style = """
<style>
    td:nth-child(7) {text-align: left;}
</style>
"""
html_table = metrics_df.to_html(index=False, max_rows=None, max_cols=None, notebook=True, escape=False)
styled_html = f"{custom_style}{html_table}"

# Display the styled HTML table
display(HTML(styled_html))


In [ ]:
import pandas as pd
from IPython.display import display, HTML
import matplotlib.pyplot as plt

# Ensure the expected column exists in df
if 'expected' not in df.columns:
    raise KeyError("The 'expected' column does not exist in the DataFrame.")

# Count the occurrences of each service number in the 'expected' column
service_number_counts = df['expected'].value_counts().reset_index()
service_number_counts.columns = ['service_number', 'count']

# Render the counts as an HTML table
custom_style = """
<style>
    td:nth-child(1), td:nth-child(2) {text-align: left;} /* Align service number and count to the left */
</style>
"""
html_table = service_number_counts.to_html(index=False, max_rows=None, max_cols=None, notebook=True, escape=False)
styled_html = f"{custom_style}{html_table}"

# Display the table
print("\nMost Frequent Service Numbers in the 'expected' Column:")
display(HTML(styled_html))

# Plot the top 10 most frequent service numbers
top_10 = service_number_counts.head(10)
plt.figure(figsize=(10, 6))
plt.bar(top_10['service_number'].astype(str), top_10['count'], color='skyblue', edgecolor='black', alpha=0.7)
plt.title('Top 10 Most Frequent Service Numbers', fontsize=14)
plt.xlabel('Service Number', fontsize=12)
plt.ylabel('Frequency', fontsize=12)
plt.xticks(rotation=45)
plt.tight_layout()
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()


In [ ]:
import pandas as pd
from difflib import SequenceMatcher
from collections import defaultdict
from IPython.display import display, HTML

# Function to find similarity between strings
def find_common_prefix(strings, min_length=10):
    """Find the common prefix across a list of strings."""
    if not strings:
        return ""
    common_prefix = strings[0]
    for s in strings[1:]:
        matcher = SequenceMatcher(None, common_prefix, s)
        match = matcher.find_longest_match(0, len(common_prefix), 0, len(s))
        if match.size < min_length:
            return ""
        common_prefix = common_prefix[match.a: match.a + match.size]
    return common_prefix.strip()

# Input service number to analyze
service_number_to_analyze = input("Enter the service number to analyze: ").strip()

if not service_number_to_analyze.isdigit():
    print("Invalid input! Please enter a valid numeric service number.")
else:
    # Filter the records to contain only those with the service number in the 'expected' column
    filtered_records = df[df['expected'] == service_number_to_analyze]
    
    if filtered_records.empty:
        print(f"No records found for Service Number: {service_number_to_analyze}")
    else:
        print(f"\nAnalyzing query patterns for Service Number: {service_number_to_analyze}\n")

        # Group queries based on their front portion similarity
        query_groups = defaultdict(list)
        queries = filtered_records['query'].tolist()
        
        for query in queries:
            grouped = False
            for group_key, group_list in query_groups.items():
                if SequenceMatcher(None, query, group_key).ratio() >= 0.8:
                    group_list.append(query)
                    grouped = True
                    break
            if not grouped:
                query_groups[query] = [query]

        # Summarize groups with common prefix and count
        summarized_groups = []
        for group_key, group_queries in query_groups.items():
            common_prefix = find_common_prefix(group_queries)
            summarized_groups.append({
                "Common Prefix": common_prefix or group_key[:30],  # Fallback to part of the first query
                "Query Count": len(group_queries)
            })

        grouped_df = pd.DataFrame(summarized_groups).sort_values(by="Query Count", ascending=False)

        # Render as an HTML table
        custom_style = """
        <style>
            td:nth-child(1), td:nth-child(2) {text-align: left;} /* Align columns to the left */
        </style>
        """
        html_table = grouped_df.to_html(index=False, max_rows=None, max_cols=None, notebook=True, escape=False)
        styled_html = f"{custom_style}{html_table}"

        display(HTML(styled_html))


Insights:<br>

**High-Performing Service Numbers:**

Service Numbers 212901 (Rental-Other Assets) and 215201 (Travel & Transport-Local) stand out with high precision values of 0.929 and 0.897, respectively.
These service numbers have relatively low hit_rate, suggesting that while they appear less frequently in the top 5 matches, they are usually correct when they do appear.
High precision with low hit_rate could indicate strong alignment between these service numbers and specific query types.<br>

**Moderate Precision with High Hit Rate:**

217501 (Tech Services-ICT Security/Audit Services) and 213201 (Service for Meals & Refreshments) have higher hit_rate but moderate precision (0.528 and 0.440, respectively).
This suggests these service numbers are more frequently retrieved but are less consistently correct.
They might be over-generalized and retrieved for queries they don't align well with.<br>


**Low Precision with High Hit Rate:**

Low precision in service numbers like 218401 (TOther Services-Postage and Courier Services) suggests potential issues with model embeddings or similarity thresholds.
They may be retrieved for queries that align poorly with their descriptions.

**Query analysis of selected service number**

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
import re
from IPython.display import display, HTML

pd.set_option('display.max_colwidth', None)

# Function to clean and extract meaningful keywords
def extract_keywords(text):
    """Extract meaningful keywords by removing stop words and non-alphabetic characters."""
    words = re.findall(r'\b\w+\b', text.lower())  # Extract words
    meaningful_words = [word for word in words if word not in ENGLISH_STOP_WORDS]  # Exclude stop words
    return meaningful_words

# Input the service number
service_number_to_analyze = input("Enter the service number to analyze: ").strip()

if not service_number_to_analyze.isdigit():
    print("Invalid input! Please enter a valid numeric service number.")
else:
    service_queries = df_copy[df_copy['retrieved_material_number'] == service_number_to_analyze]
    
    if service_queries.empty:
        print(f"No queries found for Service Number: {service_number_to_analyze}")
    else:
        # Calculate query lengths
        service_queries['query_length'] = service_queries['query'].str.len()

        # Calculate metrics
        total_correct = service_queries['is_correct'].sum()
        total_incorrect = len(service_queries) - total_correct
        accuracy = total_correct / len(service_queries) if len(service_queries) > 0 else 0

        # Display overall metrics
        print(f"Total Correct: {total_correct}")
        print(f"Total Incorrect: {total_incorrect}")
        print(f"Accuracy: {accuracy:.3f}\n")

        # Sort by correctness and similarity score
        service_queries = service_queries.sort_values(by=['is_correct', 'score'], ascending=[True, False])
        service_query_details = service_queries[['query', 'query_length', 'expected_description', 
                                                 'retrieved_description', 'score', 'is_correct']]

        # Render table with custom CSS for alignment
        custom_style = """
        <style>
            td:nth-child(1), td:nth-child(3), td:nth-child(4) {text-align: left;} /* Align text columns to the left */
        </style>
        """
        html_table = service_query_details.to_html(index=False, max_rows=None, max_cols=None, notebook=True, escape=False)
        styled_html = f"{custom_style}{html_table}"
        
        # Plot similarity score distribution
        plt.hist(service_queries['score'], bins=range(0, 101, 5), color='skyblue', edgecolor='black', alpha=0.7)
        plt.title(f"Similarity Score Distribution for Service Number {service_number_to_analyze}", fontsize=14)
        plt.xlabel("Similarity Score", fontsize=12)
        plt.ylabel("Frequency", fontsize=12)
        plt.grid(axis='y', linestyle='--', alpha=0.7)
        plt.xlim(0, 100)  
        plt.tight_layout()
        plt.show()
        
        # Plot query lengths vs. similarity scores
        plt.figure(figsize=(10, 6))
        plt.scatter(service_queries[service_queries['is_correct']]['query_length'],
                    service_queries[service_queries['is_correct']]['score'],
                    color='blue', label='Correct', alpha=0.7, edgecolors='k')
        plt.scatter(service_queries[~service_queries['is_correct']]['query_length'],
                    service_queries[~service_queries['is_correct']]['score'],
                    color='red', label='Incorrect', alpha=0.7, edgecolors='k')

        # Add legend and labels
        plt.title(f"Query Length vs. Similarity Score for Service Number {service_number_to_analyze}", fontsize=14)
        plt.xlabel("Query Length", fontsize=12)
        plt.ylabel("Similarity Score", fontsize=12)
        plt.legend(title="Match Status", fontsize=10, loc="upper right")
        plt.grid(alpha=0.5)
        plt.tight_layout()
        plt.show()
        
        # Keyword Analysis
        correct_queries = service_queries[service_queries['is_correct']]['query'].apply(lambda q: set(extract_keywords(q)))
        incorrect_queries = service_queries[~service_queries['is_correct']]['query'].apply(lambda q: set(extract_keywords(q)))

        # Flatten the sets and count frequencies
        correct_keywords = Counter([kw for query_keywords in correct_queries for kw in query_keywords])
        incorrect_keywords = Counter([kw for query_keywords in incorrect_queries for kw in query_keywords])

        # Create a DataFrame for keyword comparison
        keywords_df = pd.DataFrame({
            "keyword": list(set(correct_keywords.keys()).union(set(incorrect_keywords.keys()))),
            "correct_count": [correct_keywords.get(k, 0) for k in set(correct_keywords.keys()).union(set(incorrect_keywords.keys()))],
            "incorrect_count": [incorrect_keywords.get(k, 0) for k in set(correct_keywords.keys()).union(set(incorrect_keywords.keys()))]
        }).sort_values(by="correct_count", ascending=False)

        # Display keyword analysis table
        service_description = service_queries["retrieved_description"].iloc[0]  # Get the description of the service number
        print(f"\nKeyword Analysis for Service Number: {service_number_to_analyze}")
        print(f"Description: {service_description}\n")
        display(HTML(keywords_df.head(20).to_html(index=False, max_rows=None)))

        # Plot top 10 meaningful keywords for correct and incorrect queries
        top_keywords = keywords_df.head(10)
        plt.figure(figsize=(10, 6))
        bar_width = 0.35
        index = range(len(top_keywords))

        plt.bar(index, top_keywords['correct_count'], bar_width, label='Correct', color='blue', alpha=0.7)
        plt.bar([i + bar_width for i in index], top_keywords['incorrect_count'], bar_width, label='Incorrect', color='red', alpha=0.7)

        plt.xlabel('Keywords', fontsize=12)
        plt.ylabel('Frequency', fontsize=12)
        plt.title(f"Top Keywords for Service Number {service_number_to_analyze}", fontsize=14)
        plt.xticks([i + bar_width / 2 for i in index], top_keywords['keyword'], rotation=45)
        plt.legend()
        plt.tight_layout()
        plt.show()

        # Display the table
        print(f"\nAnalysis of Queries for Service Number: {service_number_to_analyze}")
        display(HTML(styled_html))


**Code below to export results to excel**

In [23]:
# # Prepare data for export
# additional_data = []

# for result in results:  
#     # Combine expected material number and description
#     expected_combined = f"{result['expected']} - {result['expected_description']}"

#     # Combine top 5 matched material numbers into a single string
#     top_matches_combined = "; ".join(
#         [f"{match['material_number']} score:{match['score']}" for match in result["retrieved_top_5"]]
#     )

#     additional_data.append({
#         "query": result["query"],
#         "expected": expected_combined,  
#         "matches": top_matches_combined,  
#         "is_correct": "True" if result["is_correct"] else "False"  
#     })

# additional_df = pd.DataFrame(additional_data)

# output_path = "../data/output/evaluation_results.xlsx"

# # Dynamically get the model name
# model_name = getattr(search_engine, "model_name", "Unknown_Model")
# sheet_name = model_name.replace("/", "_")[:31]  

# os.makedirs(os.path.dirname(output_path), exist_ok=True)

# # Append to the Excel file
# with pd.ExcelWriter(output_path, engine="openpyxl", mode="a", if_sheet_exists="replace") as writer:
#     additional_df.to_excel(writer, index=False, sheet_name=sheet_name)

# print(f"Additional results appended to sheet '{sheet_name}' in: {output_path}")

# 